# Ok, Let's go

### Import Libraries and Modules

In [58]:
# Install required packages
!pip install numpy pandas beautifulsoup4 requests matplotlib seaborn tqdm ipython recordlinkage jellyfish numexpr missingno tyrone_mings lxml

  Using cached tyrone_mings-1.3.0-py2.py3-none-any.whl.metadata (665 bytes)
  Using cached lxml-5.3.1-cp313-cp313-win_amd64.whl.metadata (3.8 kB)
  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached js2xml-0.3.1-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached slimit-0.8.1.zip (88 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      Python 3.X support requires the 2to3 tool.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [59]:
# Python ≥3.5 (ideally)
import platform
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd
import os
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os
from os.path import basename

# Flatten lists
from functools import reduce

# Working with JSON
import json
from pandas import json_normalize  # Updated import

# Web Scraping
import requests
from bs4 import BeautifulSoup
import re

# APIs
# from tyrone_mings import *  # Comment out this custom module for now

# Fuzzy Matching - Record Linkage
import recordlinkage
import jellyfish
import numexpr as ne

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-whitegrid')  # Update


In [60]:
# Python / module versions used here for reference
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))

Python: 3.13.2
NumPy: 2.2.4
pandas: 2.2.3
matplotlib: 3.10.1


In [66]:
import os
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\Andres Pascoe\Downloads


In [ ]:
import os
print(os.listdir("a_league_data"))  # check if CSVs are in this subfolder

['playing_time.csv', 'shooting_stats.csv', 'standard_stats.csv']


In [4]:
!pip install dash

In [5]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("a_league_processed/standard_stats_processed.csv")

# Convert 'Min' column to string first, then remove commas
df['Min'] = df['Min'].astype(str).str.replace(',', '').astype(float)

# Save the modified DataFrame back to a CSV
df.to_csv("a_league_processed/standard_stats_processed.csv", index=False)

print("CSV file updated successfully. Commas removed from 'Min' column.")

CSV file updated successfully. Commas removed from 'Min' column.



# V1

In [12]:
import dash
from dash import dcc, html, dash_table
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import webbrowser
from threading import Timer

# Load the already processed data
df = pd.read_csv("a_league_processed/standard_stats_processed.csv")

# Create a dictionary of team colors (based on real A-League team colors)
team_colors = {
    'Adelaide': '#9b2633',  # dimmed red
    'Auckland FC': '#145a7d',  # muted teal blue
    'Brisbane': '#b76d10',  # muted orange
    'Central Coast': '#d0af4c',  # softened yellow
    'Macarthur FC': '#313131',  # dark gray/black
    'Melb City': '#6295c3',  # sky blue
    'Melb Victory': '#1a3a7c',  # deep navy blue
    'Newcastle': '#d1a03e',  # gold (using only the main color)
    'Perth Glory': '#4f2c7d',  # muted purple
    'Sydney FC': '#4b77b8',  # dimmed blue
    'W Sydney': '#bc3034',  # muted red
    'Wellington': '#c3a023',  # toned-down yellow
    'Western United': '#2d5234',  # dimmed green
    'Western Sydney Wanderers': '#bc3034'  # muted red
}

# Create the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Custom CSS to match your design
app.layout = html.Div([
    html.H1("A-League Mens - Under 23 Player Analysis",
            style={'textAlign': 'center', 'color': '#3182ce', 'marginBottom': '30px', 'fontSize': '28px', 'fontWeight': '700'}),
    
    # Tabs
    html.Div([
        html.Div("Overview", id="tab-overview", className="tab active", **{'data-tab': 'overview'}),
        html.Div("Player Market Value [wip]", id="tab-scorers", className="tab", **{'data-tab': 'scorers'}),
        html.Div("On-Ball Value (OBV) [wip]", id="tab-assisters", className="tab", **{'data-tab': 'assisters'}),
        html.Div("Passing Stats [wip]", id="tab-minutes", className="tab", **{'data-tab': 'minutes'}),
        html.Div("Shooting Stats [wip]", id="tab-teams", className="tab", **{'data-tab': 'teams'}),
    ], className="tabs"),
    
    # Overview Tab Content
    html.Div([
        # Summary Stats
        html.Div([
            html.Div([
                html.Div("Total Players", className="metric-title"),
                html.Div(id="total-players", className="metric-value"),
                html.Div("Total Players Under 23", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}), # Added U23
                html.Div(id="total-players-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}), # Added U23
            ], className="metric-card"),

            html.Div([
                html.Div("Total Goals", className="metric-title"),
                html.Div(id="total-goals", className="metric-value"),
                html.Div("Total Goals Under 23", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}), # Added U23
                html.Div(id="total-goals-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}), # Added U23
            ], className="metric-card"),

            html.Div([
                html.Div("Total Assists", className="metric-title"),
                html.Div(id="total-assists", className="metric-value"),
                html.Div("Total Assists Under 23", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}), # Added U23
                html.Div(id="total-assists-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}), # Added U23
            ], className="metric-card"),

            html.Div([
                html.Div("Total Minutes", className="metric-title"),
                html.Div(id="total-minutes", className="metric-value"),
                html.Div("Total Minutes Under 23", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}), # Added U23
                html.Div(id="total-minutes-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}), # Added U23
            ], className="metric-card"),
        ], className="metrics-row"),
        
        # Charts
        html.Div([
            html.Div([
                dcc.Graph(id="goals-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="assisters-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="minutes-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="teams-chart")
            ], className="chart-container"),
        ], className="dashboard-container"),
        
        # Filters
        html.Div([
            html.Div([
                html.Label("Position:", style={'marginRight': '10px'}),
                dcc.Dropdown(
                    id="position-filter",
                    options=[{"label": "All Positions", "value": "all"}] +
                            [{"label": pos, "value": pos} for pos in df["Pos"].unique() if pd.notna(pos)],
                    value="all",
                    style={'width': '200px'}
                ),
                
                html.Label("Min Minutes:", style={'marginLeft': '15px', 'marginRight': '10px'}),
                dcc.Dropdown(
                    id="min-minutes-filter",
                    options=[
                        {"label": "All", "value": 0},
                        {"label": "90+", "value": 90},
                        {"label": "270+", "value": 270},
                        {"label": "450+", "value": 450},
                        {"label": "900+", "value": 900},
                    ],
                    value=0,
                    style={'width': '150px'}
                ),
                html.Label("Under 23 Only:", style={'marginLeft': '15px', 'marginRight': '10px'}), # Added U23 Filter
                dcc.Checklist(
                    id="u23-filter",
                    options=[
                        {"label": "Yes", "value": "yes"},
                    ],
                    value=[],
                    style={'width': '50px'}
                ),
            ], className="filter-group"),
            
            html.Div([
                dcc.Input(
                    id="player-search",
                    type="text",
                    placeholder="Search players...",
                    style={'padding': '8px 12px', 'width': '200px', 'borderRadius': '4px', 'border': '1px solid #e2e8f0'}
                ),
            ], className="filter-group"),
        ], className="filter-container"),
        
        # Players Table
        html.Div([
            html.H2("Player Statistics"),
            dash_table.DataTable(
                id="players-table",
                style_table={'overflowX': 'auto'},
                style_header={
                    'backgroundColor': 'rgb(240, 240, 240)',
                    'fontWeight': 'bold',
                    'border': '1px solid #e2e8f0'
                },
                style_cell={
                    'padding': '10px',
                    'border': '1px solid #e2e8f0',
                    'textAlign': 'left'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 250, 252)'
                    }
                ]
            )
        ], className="table-container"),
        
    ], id="content-overview", className="tab-content active"),
    
    # Tab content for other tabs would go here
    
], style={
    'fontFamily': 'Segoe UI, Tahoma, Geneva, Verdana, sans-serif',
    'margin': '0',
    'padding': '20px',
    'backgroundColor': '#f7fafc',
    'color': '#2d3748',
})

# Add CSS
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>A-League Analytics Dashboard</title>
        {%favicon%}
        {%css%}
        <style>
            :root {
                --primary: #3182ce;
                --secondary: #e53e3e;
                --accent: #38a169;
                --background: #f7fafc;
                --card-bg: #ffffff;
                --text: #2d3748;
                --text-light: #718096;
                --border: #e2e8f0;
                --shadow: 0 4px 6px rgba(0, 0, 0, 0.05);
            }
            
            .tabs {
                display: flex;
                border-bottom: 1px solid var(--border);
                margin-bottom: 20px;
            }
            
            .tab {
                padding: 10px 20px;
                cursor: pointer;
                border-bottom: 2px solid transparent;
                font-weight: 500;
            }
            
            .tab.active {
                border-bottom: 2px solid var(--primary);
                color: var(--primary);
            }
            
            .tab-content {
                display: none;
            }
            
            .tab-content.active {
                display: block;
            }
            
            .metrics-row {
                display: grid;
                grid-template-columns: repeat(4, 1fr);
                gap: 20px;
                margin-bottom: 20px;
            }
            
            .metric-card {
                background-color: var(--card-bg);
                border-radius: 8px;
                padding: 15px;
                box-shadow: var(--shadow);
                text-align: center;
            }
            
            .metric-value {
                font-size: 24px;
                font-weight: 700;
                color: var(--primary);
                margin: 10px 0;
            }
            
            .metric-title {
                font-size: 14px;
                color: var(--text-light);
            }
            
            .dashboard-container {
                display: grid;
                grid-template-columns: repeat(2, 1fr);
                gap: 20px;
                margin-bottom: 30px;
            }
            
            .chart-container {
                background-color: var(--card-bg);
                border-radius: 8px;
                padding: 20px;
                box-shadow: var(--shadow);
            }
            
            .table-container {
                background-color: var(--card-bg);
                border-radius: 8px;
                padding: 20px;
                box-shadow: var(--shadow);
                overflow-x: auto;
            }
            
            .filter-container {
                display: flex;
                justify-content: space-between;
                align-items: center;
                margin-bottom: 15px;
                flex-wrap: wrap;
                gap: 10px;
            }
            
            .filter-group {
                display: flex;
                align-items: center;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# Function to get color for team
def get_team_color(team):
    # Default color if team not found
    default_color = '#808080'  # Gray
    return team_colors.get(team, default_color)

# Callback to update charts and tables when filters change
@app.callback(
    [
        # Summary metric outputs
        dash.dependencies.Output('total-players', 'children'),
        dash.dependencies.Output('total-goals', 'children'),
        dash.dependencies.Output('total-assists', 'children'),
        dash.dependencies.Output('total-minutes', 'children'),
        
        # Chart outputs
        dash.dependencies.Output('goals-chart', 'figure'),
        dash.dependencies.Output('assisters-chart', 'figure'),
        dash.dependencies.Output('minutes-chart', 'figure'),
        dash.dependencies.Output('teams-chart', 'figure'),
        
        # Table output
        dash.dependencies.Output('players-table', 'data'),
        dash.dependencies.Output('players-table', 'columns'),
    ],
    [
        dash.dependencies.Input('position-filter', 'value'),
        dash.dependencies.Input('min-minutes-filter', 'value'),
        dash.dependencies.Input('player-search', 'value'),
    ]
)
def update_dashboard(position, min_minutes, search_term):
    # Filter data based on inputs
    filtered_data = df.copy()
    
    # Position filter
    if position != "all" and position is not None:
        filtered_data = filtered_data[filtered_data["Pos"] == position]
    
    # Minutes filter
    if min_minutes > 0:
        filtered_data = filtered_data[filtered_data["Min"] >= min_minutes]


    
    # Search filter
    if search_term:
        filtered_data = filtered_data[filtered_data["Player"].str.contains(search_term, case=False)]
    
    # Calculate summary metrics
    total_players = len(filtered_data)
    total_goals = filtered_data["Gls"].sum()
    
    # Handle 'Ast' which might be in different columns
    assist_col = None
    for col in ['Ast', 'Ast.1']:
        if col in filtered_data.columns:
            assist_col = col
            break
            
    total_assists = filtered_data[assist_col].sum() if assist_col else 0
    total_minutes = filtered_data["Min"].sum()
    
    # Create a color mapping based on teams
    teams = filtered_data["Squad"].unique()
    color_discrete_map = {team: get_team_color(team) for team in teams}
    
    # Create charts
    # 1. Goals Chart
    top_scorers = filtered_data.nlargest(10, "Gls").sort_values("Gls")
    goals_fig = px.bar(
        top_scorers,
        x="Gls",
        y="Player",
        orientation="h",
        title="Top 10 Goal Scorers",
        color="Squad",
        color_discrete_map=color_discrete_map,
        labels={"Gls": "Goals", "Player": "Player"},
        height=400
    )
    goals_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 2. Assists Chart
    if assist_col:
        top_assisters = filtered_data.nlargest(10, assist_col).sort_values(assist_col)
        assisters_fig = px.bar(
            top_assisters,
            x=assist_col,
            y="Player",
            orientation="h",
            title="Top 10 Assist Providers",
            color="Squad",
            color_discrete_map=color_discrete_map,
            labels={assist_col: "Assists", "Player": "Player"},
            height=400
        )
    else:
        # Fallback if no assist column
        assisters_fig = go.Figure()
        assisters_fig.update_layout(title="Assist data not available")
    
    assisters_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 3. Minutes Chart
    top_minutes = filtered_data.nlargest(10, "Min").sort_values("Min")
    minutes_fig = px.bar(
        top_minutes,
        x="Min",
        y="Player",
        orientation="h",
        title="Top 10 Players by Minutes Played",
        color="Squad",
        color_discrete_map=color_discrete_map,
        labels={"Min": "Minutes Played", "Player": "Player"},
        height=400
    )
    minutes_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 4. Team Goals Chart
    team_goals = filtered_data.groupby("Squad")["Gls"].sum().reset_index().sort_values("Gls")
    teams_fig = px.bar(
        team_goals,
        x="Gls",
        y="Squad",
        orientation="h",
        title="Teams by Total Goals",
        color="Squad",
        color_discrete_map=color_discrete_map,
        labels={"Gls": "Total Goals", "Squad": "Team"},
        height=400
    )
    teams_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # Prepare table data
    clean_df = filtered_data.copy()
    if 'Min' in clean_df.columns:
        clean_df['Min'] = pd.to_numeric(clean_df['Min'], errors='coerce')
        # Replace NaN with empty string for display
        clean_df['Min'] = clean_df['Min'].fillna('').astype(str)
        # Remove decimal part if it's just .0
        clean_df['Min'] = clean_df['Min'].apply(lambda x: x.replace('.0', '') if x.endswith('.0') else x)

    table_data = clean_df.sort_values("Gls", ascending=False).to_dict('records')
    
    # Define table columns
    table_columns = [
        {"name": "Player", "id": "Player"},
        {"name": "Pos", "id": "Pos"},
        {"name": "Squad", "id": "Squad"},
        {"name": "Age", "id": "Age"},
        {"name": "MP", "id": "MP"},
        {"name": "Min", "id": "Min"},
        {"name": "Goals", "id": "Gls"}
    ]
    
    # Add assists column if it exists
    if assist_col:
        table_columns.append({"name": "Ast", "id": assist_col})
    
    # Add 90s column if it exists
    if '90s' in filtered_data.columns:
        table_columns.append({"name": "90s", "id": "90s"})
    
    return total_players, total_goals, total_assists, f"{total_minutes:,}", goals_fig, assisters_fig, minutes_fig, teams_fig, table_data, table_columns

# Function to open the browser automatically
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

# Run this after the app
if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run(debug=False)


# V2 [wip]

In [13]:
def update_dashboard(position, min_minutes, search_term, u23_filter):
    print(f"Callback triggered with filters: position={position}, min_minutes={min_minutes}, search={search_term}, u23={u23_filter}")
    
    # Filter data based on inputs
    filtered_data = merged_df.copy()
    print(f"Starting with {len(filtered_data)} rows of data")
    
    # Position filter
    if position != "all" and position is not None:
        filtered_data = filtered_data[filtered_data["Pos"] == position]
        print(f"After position filter: {len(filtered_data)} rows")
    
    # Minutes filter
    if min_minutes > 0 and "Min" in filtered_data.columns:
        filtered_data = filtered_data[filtered_data["Min"] >= min_minutes]
        print(f"After minutes filter: {len(filtered_data)} rows")
    
    # U23 filter
    is_u23_filter_active = 'yes' in u23_filter
    
    # Search filter
    if search_term:
        filtered_data = filtered_data[filtered_data["Player"].str.contains(search_term, case=False)]
        print(f"After search filter: {len(filtered_data)} rows")
    
    # Get U23 players subset - handle different possible age column formats
    u23_data = pd.DataFrame()
    
    # Try different age columns that might be present
    for age_col in ['Age', 'Age_Years']:
        if age_col in filtered_data.columns and filtered_data[age_col].notna().any():
            u23_data = filtered_data[filtered_data[age_col] < 23]
            print(f"Found {len(u23_data)} U23 players using '{age_col}' column")
            break
    
    # If no valid age columns, create an empty subset
    if u23_data.empty:
        u23_data = filtered_data.head(0)  # Empty DataFrame with same columns
        print("Warning: No U23 players found - check age data processing")
    
    # Calculate standard summary metrics
    total_players = len(filtered_data)
    total_goals = filtered_data["Gls"].sum() if "Gls" in filtered_data.columns else 0
    print(f"Total players: {total_players}, Total goals: {total_goals}")
    
    # Handle 'Ast' which might be in different columns
    assist_col = None
    for col in ['Ast', 'Ast.1']:
        if col in filtered_data.columns:
            assist_col = col
            print(f"Using assist column: {assist_col}")
            break
            
    total_assists = filtered_data[assist_col].sum() if assist_col else 0
    total_minutes = filtered_data["Min"].sum() if "Min" in filtered_data.columns else 0
    print(f"Total assists: {total_assists}, Total minutes: {total_minutes}")
    
    # U23 standard metrics
    total_players_u23 = len(u23_data)
    total_goals_u23 = u23_data["Gls"].sum() if "Gls" in u23_data.columns else 0
    total_assists_u23 = u23_data[assist_col].sum() if assist_col and assist_col in u23_data.columns else 0
    total_minutes_u23 = u23_data["Min"].sum() if "Min" in u23_data.columns else 0
    print(f"U23 players: {total_players_u23}, U23 goals: {total_goals_u23}")
    
    # Calculate shooting summary metrics
    total_shots = filtered_data["Sh"].sum() if "Sh" in filtered_data.columns else 0
    total_sot = filtered_data["SoT"].sum() if "SoT" in filtered_data.columns else 0
    print(f"Total shots: {total_shots}, Total SoT: {total_sot}")
    
    # Calculate average SoT% and Goals/Shot, filtering out NaN values
    if "SoT%" in filtered_data.columns:
        valid_sot_pct = filtered_data["SoT%"].dropna()
        avg_sot_pct = valid_sot_pct.mean() if not valid_sot_pct.empty else 0
        print(f"Avg SoT%: {avg_sot_pct:.2f}% (from {len(valid_sot_pct)} valid values)")
    else:
        avg_sot_pct = 0
        print("SoT% column not found")
        
    if "G/Sh" in filtered_data.columns:
        valid_g_per_sh = filtered_data["G/Sh"].dropna()
        avg_g_per_sh = valid_g_per_sh.mean() if not valid_g_per_sh.empty else 0
        print(f"Avg G/Sh: {avg_g_per_sh:.3f} (from {len(valid_g_per_sh)} valid values)")
    else:
        avg_g_per_sh = 0
        print("G/Sh column not found")
    
    # U23 shooting metrics
    total_shots_u23 = u23_data["Sh"].sum() if "Sh" in u23_data.columns else 0
    total_sot_u23 = u23_data["SoT"].sum() if "SoT" in u23_data.columns else 0
    print(f"U23 shots: {total_shots_u23}, U23 SoT: {total_sot_u23}")
    
    if "SoT%" in u23_data.columns:
        valid_sot_pct_u23 = u23_data["SoT%"].dropna()
        avg_sot_pct_u23 = valid_sot_pct_u23.mean() if not valid_sot_pct_u23.empty else 0
    else:
        avg_sot_pct_u23 = 0
        
    if "G/Sh" in u23_data.columns:
        valid_g_per_sh_u23 = u23_data["G/Sh"].dropna()
        avg_g_per_sh_u23 = valid_g_per_sh_u23.mean() if not valid_g_per_sh_u23.empty else 0
    else:
        avg_g_per_sh_u23 = 0
    
    # If U23 filter is active, use only U23 data for visualizations
    if is_u23_filter_active:
        viz_data = u23_data.copy()
        print("Using U23 data for visualizations")
    else:
        viz_data = filtered_data.copy()
        print("Using all players data for visualizations")
    
    # Create a color mapping based on teams
    teams = viz_data["Squad"].unique() if "Squad" in viz_data.columns else []
    color_discrete_map = {team: get_team_color(team) for team in teams}
    
    #-------------------
    # STANDARD CHARTS
    #-------------------
    
    # 1. Goals Chart
    if "Gls" in viz_data.columns:
        # Check if we have any data with goals
        if viz_data["Gls"].sum() > 0:
            top_scorers = viz_data.nlargest(10, "Gls").sort_values("Gls")
            goals_fig = px.bar(
                top_scorers,
                x="Gls",
                y="Player",
                orientation="h",
                title="Top 10 Goal Scorers",
                color="Squad" if "Squad" in top_scorers.columns else None,
                color_discrete_map=color_discrete_map,
                labels={"Gls": "Goals", "Player": "Player"},
                height=400
            )
            print(f"Created goals chart with {len(top_scorers)} players")
        else:
            goals_fig = go.Figure()
            goals_fig.add_annotation(
                text="No goal data available for the selected filters",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            goals_fig.update_layout(title="Goals Chart (No Data)")
            print("No goals data available after filtering")
    else:
        goals_fig = go.Figure()
        goals_fig.add_annotation(
            text="Goals column not found in the dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        goals_fig.update_layout(title="Goals Chart (Missing Data)")
        print("Gls column not found in the dataset")
    
    goals_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 2. Assists Chart
    if assist_col and assist_col in viz_data.columns:
        # Check if we have any assists data
        if viz_data[assist_col].sum() > 0:
            top_assisters = viz_data.nlargest(10, assist_col).sort_values(assist_col)
            assisters_fig = px.bar(
                top_assisters,
                x=assist_col,
                y="Player",
                orientation="h",
                title="Top 10 Assist Providers",
                color="Squad" if "Squad" in top_assisters.columns else None,
                color_discrete_map=color_discrete_map,
                labels={assist_col: "Assists", "Player": "Player"},
                height=400
            )
            print(f"Created assists chart with {len(top_assisters)} players")
        else:
            assisters_fig = go.Figure()
            assisters_fig.add_annotation(
                text="No assist data available for the selected filters",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            assisters_fig.update_layout(title="Assists Chart (No Data)")
            print("No assists data available after filtering")
    else:
        # Fallback if no assist column
        assisters_fig = go.Figure()
        assisters_fig.add_annotation(
            text="Assists column not found in the dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        assisters_fig.update_layout(title="Assists Chart (Missing Data)")
        print("No valid assist column found in the dataset")
    
    assisters_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 3. Minutes Chart
    if "Min" in viz_data.columns:
        # Check if we have minutes data
        if viz_data["Min"].sum() > 0:
            top_minutes = viz_data.nlargest(10, "Min").sort_values("Min")
            minutes_fig = px.bar(
                top_minutes,
                x="Min",
                y="Player",
                orientation="h",
                title="Top 10 Players by Minutes Played",
                color="Squad" if "Squad" in top_minutes.columns else None,
                color_discrete_map=color_discrete_map,
                labels={"Min": "Minutes Played", "Player": "Player"},
                height=400
            )
            print(f"Created minutes chart with {len(top_minutes)} players")
        else:
            minutes_fig = go.Figure()
            minutes_fig.add_annotation(
                text="No minutes data available for the selected filters",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            minutes_fig.update_layout(title="Minutes Chart (No Data)")
            print("No minutes data available after filtering")
    else:
        minutes_fig = go.Figure()
        minutes_fig.add_annotation(
            text="Minutes column not found in the dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        minutes_fig.update_layout(title="Minutes Chart (Missing Data)")
        print("Min column not found in the dataset")
    
    minutes_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 4. Team Goals Chart
    if "Gls" in viz_data.columns and "Squad" in viz_data.columns:
        # Check if we have any goals by team
        team_goals = viz_data.groupby("Squad")["Gls"].sum().reset_index()
        if not team_goals.empty and team_goals["Gls"].sum() > 0:
            team_goals = team_goals.sort_values("Gls")
            teams_fig = px.bar(
                team_goals,
                x="Gls",
                y="Squad",
                orientation="h",
                title="Teams by Total Goals",
                color="Squad",
                color_discrete_map=color_discrete_map,
                labels={"Gls": "Total Goals", "Squad": "Team"},
                height=400
            )
            print(f"Created team goals chart with {len(team_goals)} teams")
        else:
            teams_fig = go.Figure()
            teams_fig.add_annotation(
                text="No team goals data available for the selected filters",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            teams_fig.update_layout(title="Team Goals Chart (No Data)")
            print("No team goals data available after filtering")
    else:
        teams_fig = go.Figure()
        teams_fig.add_annotation(
            text="Goals or Team columns not found in the dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        teams_fig.update_layout(title="Team Goals Chart (Missing Data)")
        print("Gls or Squad columns not found in the dataset")
    
    teams_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    #-------------------
    # SHOOTING CHARTS
    #-------------------
    
    # 1. Total Shots Chart
    if "Sh" in viz_data.columns:
        # Check if we have shots data
        if viz_data["Sh"].sum() > 0:
            top_shooters = viz_data.nlargest(10, "Sh").sort_values("Sh")
            shots_fig = px.bar(
                top_shooters,
                x="Sh",
                y="Player",
                orientation="h",
                title="Top 10 Players by Total Shots",
                color="Squad" if "Squad" in top_shooters.columns else None,
                color_discrete_map=color_discrete_map,
                labels={"Sh": "Total Shots", "Player": "Player"},
                height=400
            )
            print(f"Created shots chart with {len(top_shooters)} players")
        else:
            shots_fig = go.Figure()
            shots_fig.add_annotation(
                text="No shots data available for the selected filters",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            shots_fig.update_layout(title="Shots Chart (No Data)")
            print("No shots data available after filtering")
    else:
        shots_fig = go.Figure()
        shots_fig.add_annotation(
            text="Shots column not found in the dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        shots_fig.update_layout(title="Shots Chart (Missing Data)")
        print("Sh column not found in the dataset")
    
    shots_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 2. Shot on Target % Chart
    if "SoT%" in viz_data.columns and "Sh" in viz_data.columns:
        # Filter players with at least 5 shots for better visualization
        min_shots = 5
        qualified_players = viz_data[viz_data["Sh"] >= min_shots]
        
        # Check if we have qualified players
        if not qualified_players.empty and qualified_players["SoT%"].notna().any():
            top_accuracy = qualified_players.nlargest(10, "SoT%").sort_values("SoT%")
            sot_pct_fig = px.bar(
                top_accuracy,
                x="SoT%",
                y="Player",
                orientation="h",
                title=f"Top 10 Players by Shot on Target % (min. {min_shots} shots)",
                color="Squad" if "Squad" in top_accuracy.columns else None,
                color_discrete_map=color_discrete_map,
                labels={"SoT%": "Shot on Target %", "Player": "Player"},
                height=400
            )
            print(f"Created SoT% chart with {len(top_accuracy)} qualified players")
        else:
            sot_pct_fig = go.Figure()
            sot_pct_fig.add_annotation(
                text=f"No players with {min_shots}+ shots meet the criteria",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            sot_pct_fig.update_layout(title="Shot on Target % Chart (Insufficient Data)")
            print(f"No players with {min_shots}+ shots available after filtering")
    else:
        sot_pct_fig = go.Figure()
        sot_pct_fig.add_annotation(
            text="Shot on Target % column not found in the dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        sot_pct_fig.update_layout(title="Shot on Target % Chart (Missing Data)")
        print("SoT% or Sh columns not found in the dataset")
    
    sot_pct_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 3. Shots per 90 Chart
    if "Sh/90" in viz_data.columns and "90s" in viz_data.columns:
        # Filter players with minimum minutes for better reliability
        min_90s = 3  # At least 3 full matches worth of minutes
        qualified_players = viz_data[viz_data["90s"] >= min_90s]
        
        # Check if we have qualified players
        if not qualified_players.empty and qualified_players["Sh/90"].notna().any():
            top_volume = qualified_players.nlargest(10, "Sh/90").sort_values("Sh/90")
            shots_per_90_fig = px.bar(
                top_volume,
                x="Sh/90",
                y="Player",
                orientation="h",
                title=f"Top 10 Players by Shots per 90 (min. {min_90s} matches)",
                color="Squad" if "Squad" in top_volume.columns else None,
                color_discrete_map=color_discrete_map,
                labels={"Sh/90": "Shots per 90", "Player": "Player"},
                height=400
            )
            print(f"Created Sh/90 chart with {len(top_volume)} qualified players")
        else:
            shots_per_90_fig = go.Figure()
            shots_per_90_fig.add_annotation(
                text=f"No players with {min_90s}+ matches meet the criteria",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            shots_per_90_fig.update_layout(title="Shots per 90 Chart (Insufficient Data)")
            print(f"No players with {min_90s}+ matches available after filtering")
    else:
        shots_per_90_fig = go.Figure()
        shots_per_90_fig.add_annotation(
            text="Shots per 90 column not found in the dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        shots_per_90_fig.update_layout(title="Shots per 90 Chart (Missing Data)")
        print("Sh/90 or 90s columns not found in the dataset")
    
    shots_per_90_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    # 4. Goals per Shot Chart (Conversion Rate)
    if "G/Sh" in viz_data.columns and "Sh" in viz_data.columns:
        # Filter players with minimum shots for better reliability
        min_shots = 10
        qualified_players = viz_data[viz_data["Sh"] >= min_shots]
        
        # Check if we have qualified players with G/Sh data
        if not qualified_players.empty and qualified_players["G/Sh"].notna().any():
            top_converters = qualified_players.nlargest(10, "G/Sh").sort_values("G/Sh")
            g_per_shot_fig = px.bar(
                top_converters,
                x="G/Sh",
                y="Player",
                orientation="h",
                title=f"Top 10 Players by Goal Conversion Rate (min. {min_shots} shots)",
                color="Squad" if "Squad" in top_converters.columns else None,
                color_discrete_map=color_discrete_map,
                labels={"G/Sh": "Goals per Shot", "Player": "Player"},
                height=400
            )
            print(f"Created G/Sh chart with {len(top_converters)} qualified players")
        else:
            g_per_shot_fig = go.Figure()
            g_per_shot_fig.add_annotation(
                text=f"No players with {min_shots}+ shots meet the criteria",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            g_per_shot_fig.update_layout(title="Goals per Shot Chart (Insufficient Data)")
            print(f"No players with {min_shots}+ shots available after filtering")
    else:
        g_per_shot_fig = go.Figure()
        g_per_shot_fig.add_annotation(
            text="Goals per Shot column not found in the dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        g_per_shot_fig.update_layout(title="Goals per Shot Chart (Missing Data)")
        print("G/Sh or Sh columns not found in the dataset")
    
    g_per_shot_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    
    #-------------------
    # TABLES
    #-------------------
    
    # Prepare standard stats table data
    std_clean_df = viz_data.copy()
    if 'Min' in std_clean_df.columns:
        std_clean_df['Min'] = pd.to_numeric(std_clean_df['Min'], errors='coerce')
        # Replace NaN with empty string for display
        std_clean_df['Min'] = std_clean_df['Min'].fillna('').astype(str)
        # Remove decimal part if it's just .0
        std_clean_df['Min'] = std_clean_df['Min'].apply(lambda x: x.replace('.0', '') if str(x).endswith('.0') else x)
    
    # Sort by goals if available
    sort_col = "Gls" if "Gls" in std_clean_df.columns else std_clean_df.columns[0]
    std_table_data = std_clean_df.sort_values(sort_col, ascending=False).to_dict('records')
    print(f"Standard stats table: {len(std_table_data)} rows")
    
    # Define standard table columns
    std_table_columns = []
    for col_name, display_name in [
        ("Player", "Player"), 
        ("Pos", "Pos"), 
        ("Squad", "Squad"), 
        ("Age", "Age"), 
        ("MP", "MP"), 
        ("Min", "Min"), 
        ("Gls", "Goals")
    ]:
        if col_name in std_clean_df.columns:
            std_table_columns.append({"name": display_name, "id": col_name})
    
    # Add assists column if it exists
    if assist_col and assist_col in std_clean_df.columns:
        std_table_columns.append({"name": "Ast", "id": assist_col})
    
    # Add 90s column if it exists
    if '90s' in std_clean_df.columns:
        std_table_columns.append({"name": "90s", "id": "90s"})
    
    print(f"Standard stats table columns: {[col['name'] for col in std_table_columns]}")
    
    # Prepare shooting stats table data
    shoot_clean_df = viz_data.copy()
    
    # Sort by total shots if available
    sort_col = "Sh" if "Sh" in shoot_clean_df.columns else shoot_clean_df.columns[0]
    shoot_table_data = shoot_clean_df.sort_values(sort_col, ascending=False).to_dict('records')
    print(f"Shooting stats table: {len(shoot_table_data)} rows")
    
    # Define shooting table columns
    shoot_table_columns = []
    for col_name, display_name in [
        ("Player", "Player"),
        ("Squad", "Squad"),
        ("Pos", "Position"),
        ("90s", "90s"),
        ("Gls", "Goals"),
        ("Sh", "Shots"),
        ("SoT", "Shots on Target"),
        ("SoT%", "SoT%"),
        ("Sh/90", "Sh/90"),
        ("SoT/90", "SoT/90"),
        ("G/Sh", "Goals/Shot"),
        ("G/SoT", "Goals/SoT"),
        ("Dist", "Avg Shot Distance"),
        ("PK", "Penalties"),
    ]:
        if col_name in shoot_clean_df.columns:
            shoot_table_columns.append({"name": display_name, "id": col_name})
    
    print(f"Shooting stats table columns: {[col['name'] for col in shoot_table_columns]}")
    
    # Format metrics for display
    total_shots_formatted = f"{total_shots:,}" if pd.notna(total_shots) else "0"
    total_sot_formatted = f"{total_sot:,}" if pd.notna(total_sot) else "0"
    avg_sot_pct_formatted = f"{avg_sot_pct:.1f}%" if pd.notna(avg_sot_pct) else "0%"
    avg_g_per_sh_formatted = f"{avg_g_per_sh:.3f}" if pd.notna(avg_g_per_sh) else "0"
    
    total_shots_u23_formatted = f"{total_shots_u23:,}" if pd.notna(total_shots_u23) else "0"
    total_sot_u23_formatted = f"{total_sot_u23:,}" if pd.notna(total_sot_u23) else "0"
    avg_sot_pct_u23_formatted = f"{avg_sot_pct_u23:.1f}%" if pd.notna(avg_sot_pct_u23) else "0%"
    avg_g_per_sh_u23_formatted = f"{avg_g_per_sh_u23:.3f}" if pd.notna(avg_g_per_sh_u23) else "0"
    
    # Format standard metrics
    total_players_formatted = f"{total_players}"
    total_goals_formatted = f"{total_goals}"
    total_assists_formatted = f"{total_assists}" 
    total_minutes_formatted = f"{total_minutes:,}" if pd.notna(total_minutes) else "0"
    
    total_players_u23_formatted = f"{total_players_u23}"
    total_goals_u23_formatted = f"{total_goals_u23}"
    total_assists_u23_formatted = f"{total_assists_u23}"
    total_minutes_u23_formatted = f"{total_minutes_u23:,}" if pd.notna(total_minutes_u23) else "0"
    
    # Debug info
    debug_standard = f"Standard data rows: {len(df_standard)}, columns: {len(df_standard.columns)}"
    debug_shooting = f"Shooting data rows: {len(df_shooting)}, columns: {len(df_shooting.columns)}"
    debug_merged = f"Merged data rows: {len(merged_df)}, filtered rows: {len(filtered_data)}"
    
    print("Returning dashboard data")
    
    return (
        # Standard stats summary
        total_players_formatted, total_goals_formatted, total_assists_formatted, total_minutes_formatted,
        total_players_u23_formatted, total_goals_u23_formatted, total_assists_u23_formatted, total_minutes_u23_formatted,
        
        # Shooting stats summary
        total_shots_formatted, total_sot_formatted, avg_sot_pct_formatted, avg_g_per_sh_formatted,
        total_shots_u23_formatted, total_sot_u23_formatted, avg_sot_pct_u23_formatted, avg_g_per_sh_u23_formatted,
        
        # Standard charts
        goals_fig, assisters_fig, minutes_fig, teams_fig,
        
        # Shooting charts
        shots_fig, sot_pct_fig, shots_per_90_fig, g_per_shot_fig,
        
        # Tables
        std_table_data, std_table_columns,
        shoot_table_data, shoot_table_columns,
        
        # Debug info
        debug_standard, debug_shooting, debug_merged
    )

import dash
from dash import dcc, html, dash_table
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import webbrowser
from threading import Timer

# Add debug prints to trace data loading and processing
print("Starting dashboard initialization...")

# Load the already processed data
try:
    df_standard = pd.read_csv("a_league_processed/standard_stats_processed.csv")
    print(f"Standard stats loaded successfully: {len(df_standard)} rows")
    print(f"Standard stats columns: {df_standard.columns.tolist()}")
except Exception as e:
    print(f"Error loading standard stats: {e}")
    df_standard = pd.DataFrame()

try:
    # Keep the updated path
    df_shooting = pd.read_csv("a_league_data/shooting_stats.csv")
    print(f"Shooting stats loaded successfully: {len(df_shooting)} rows")
    print(f"Shooting stats columns: {df_shooting.columns.tolist()}")
except Exception as e:
    print(f"Error loading shooting stats: {e}")
    df_shooting = pd.DataFrame()

# Create a dictionary of team colors (based on real A-League team colors)
team_colors = {
    'Adelaide': '#9b2633',  # dimmed red
    'Auckland FC': '#145a7d',  # muted teal blue
    'Brisbane': '#b76d10',  # muted orange
    'Central Coast': '#d0af4c',  # softened yellow
    'Macarthur FC': '#313131',  # dark gray/black
    'Melb City': '#6295c3',  # sky blue
    'Melb Victory': '#1a3a7c',  # deep navy blue
    'Newcastle': '#d1a03e',  # gold (using only the main color)
    'Perth Glory': '#4f2c7d',  # muted purple
    'Sydney FC': '#4b77b8',  # dimmed blue
    'W Sydney': '#bc3034',  # muted red
    'Wellington': '#c3a023',  # toned-down yellow
    'Western United': '#2d5234',  # dimmed green
    'Western Sydney Wanderers': '#bc3034'  # muted red
}

# Process the shooting data
def process_shooting_data(df):
    if df.empty:
        print("Warning: Shooting data is empty")
        return df
        
    print("Processing shooting data...")
    
    # Skip the first row if it contains headers
    if len(df) > 0 and df.iloc[0].values[0] == 'Standard':
        print("Detected header row in shooting stats, skipping...")
        df = df.iloc[1:].reset_index(drop=True)
    
    # Print first few rows to debug
    print("First 3 rows of shooting data after processing header:")
    print(df.head(3))
    
    # Convert numerical columns to proper data types
    numeric_cols = ['90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'PK', 'PKatt']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Process age column to extract years
    if 'Age' in df.columns:
        print("Processing Age column in shooting data...")
        df['Age_raw'] = df['Age'].copy()
        # First check if the Age column contains hyphenated values
        if df['Age'].astype(str).str.contains('-').any():
            print("Age column contains hyphenated values, extracting years...")
            df['Age'] = df['Age'].astype(str).str.split('-').str[0]
        df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
        
        # Check if Age processing was successful
        print(f"Age column after processing - min: {df['Age'].min()}, max: {df['Age'].max()}, null count: {df['Age'].isnull().sum()}")
        
        # Create U23 flag based on Age
        df['U23'] = df['Age'] < 23
        u23_count = df['U23'].sum()
        print(f"Found {u23_count} U23 players in shooting data")
    else:
        print("No Age column found in shooting data")
        # If Age column doesn't exist, add placeholder columns
        df['Age'] = None
        df['U23'] = False
    
    return df

# Process and clean the shooting data
try:
    df_shooting = process_shooting_data(df_shooting)
    print(f"Shooting data processed: {len(df_shooting)} rows")
except Exception as e:
    print(f"Error processing shooting data: {e}")
    df_shooting = pd.DataFrame()

# Merge datasets on Player and Squad if possible
def merge_datasets(standard_df, shooting_df):
    print("Attempting to merge datasets...")
    if standard_df.empty:
        print("Warning: Standard stats dataframe is empty, returning shooting stats only")
        return shooting_df
    
    if shooting_df.empty:
        print("Warning: Shooting stats dataframe is empty, returning standard stats only")
        return standard_df
    
    # Create copies to avoid modifying originals
    std_df = standard_df.copy()
    shoot_df = shooting_df.copy()
    
    # Print merge columns
    print(f"Standard df columns: {std_df.columns.tolist()}")
    print(f"Shooting df columns: {shoot_df.columns.tolist()}")
    
    # Ensure Player column exists in both
    if 'Player' in std_df.columns and 'Player' in shoot_df.columns:
        # Define merge columns
        merge_cols = ['Player']
        print(f"Common merge column: 'Player'")
        
        # Print sample values to debug matching
        print("Sample Player values in standard data:", std_df['Player'].head(3).tolist())
        print("Sample Player values in shooting data:", shoot_df['Player'].head(3).tolist())
        
        # Add Squad if present in both
        if 'Squad' in std_df.columns and 'Squad' in shoot_df.columns:
            merge_cols.append('Squad')
            print(f"Additional merge column: 'Squad'")
            
            # Print sample values to debug matching
            print("Sample Squad values in standard data:", std_df['Squad'].head(3).tolist())
            print("Sample Squad values in shooting data:", shoot_df['Squad'].head(3).tolist())
        
        # Suffix for shooting stats
        print(f"Merging on columns: {merge_cols}")
        merged_df = pd.merge(std_df, shoot_df, on=merge_cols, how='outer', suffixes=('', '_shooting'))
        print(f"Merge result: {len(merged_df)} rows")
        
        # Handle duplicated columns (like 'Gls')
        dup_cols = [col for col in shoot_df.columns if col in std_df.columns and col not in merge_cols]
        print(f"Handling duplicated columns: {dup_cols}")
        
        for col in dup_cols:
            # Keep the standard data version but fill missing values from shooting data
            if col in merged_df.columns and f"{col}_shooting" in merged_df.columns:
                merged_df[col] = merged_df[col].combine_first(merged_df[f"{col}_shooting"])
                # Drop the duplicated column
                merged_df.drop(f"{col}_shooting", axis=1, inplace=True)
        
        return merged_df
    else:
        missing_cols = []
        if 'Player' not in std_df.columns:
            missing_cols.append("'Player' in standard data")
        if 'Player' not in shoot_df.columns:
            missing_cols.append("'Player' in shooting data")
        print(f"Merge failed: Missing {', '.join(missing_cols)}")
        
        # Return a concatenated dataframe as fallback
        print("Fallback: Using standard stats only")
        return std_df

# Attempt to merge the datasets
try:
    merged_df = merge_datasets(df_standard, df_shooting)
    print(f"Final merged dataframe: {len(merged_df)} rows, {len(merged_df.columns)} columns")
    # Check if any key columns are missing
    key_cols = ['Player', 'Squad', 'Pos', 'Age', 'Gls', 'Sh', 'SoT']
    missing_key_cols = [col for col in key_cols if col not in merged_df.columns]
    if missing_key_cols:
        print(f"Warning: Missing key columns in merged data: {missing_key_cols}")
except Exception as e:
    print(f"Error merging datasets: {e}")
    # Fallback to using standard data only
    merged_df = df_standard.copy()

# Create the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Custom CSS to match your design
app.layout = html.Div([
    html.H1("A-League Mens - Under 23 Player Analysis",
            style={'textAlign': 'center', 'color': '#3182ce', 'marginBottom': '30px', 'fontSize': '28px', 'fontWeight': '700'}),
    
    # Debug info
    html.Div([
        html.H3("Debug Information", style={'color': 'red'}),
        html.P(f"Standard data rows: {len(df_standard)}", id="debug-standard"),
        html.P(f"Shooting data rows: {len(df_shooting)}", id="debug-shooting"),
        html.P(f"Merged data rows: {len(merged_df)}", id="debug-merged"),
        html.Button("Remove Debug Info", id="remove-debug", style={'margin': '10px'})
    ], id="debug-container", style={'border': '1px solid red', 'padding': '10px', 'margin-bottom': '20px'}),
    
    # Tabs
    html.Div([
        html.Div("Overview", id="tab-overview", className="tab active", **{'data-tab': 'overview'}),
        html.Div("Player Market Value [wip]", id="tab-scorers", className="tab", **{'data-tab': 'scorers'}),
        html.Div("On-Ball Value (OBV) [wip]", id="tab-assisters", className="tab", **{'data-tab': 'assisters'}),
        html.Div("Passing Stats [wip]", id="tab-minutes", className="tab", **{'data-tab': 'minutes'}),
        html.Div("Shooting Stats", id="tab-shooting", className="tab", **{'data-tab': 'shooting'}),
    ], className="tabs"),
    
    # Overview Tab Content
    html.Div([
        # Summary Stats
        html.Div([
            html.Div([
                html.Div("Total Players", className="metric-title"),
                html.Div(id="total-players", className="metric-value"),
                html.Div("Total Players Under 23", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}),
                html.Div(id="total-players-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}),
            ], className="metric-card"),

            html.Div([
                html.Div("Total Goals", className="metric-title"),
                html.Div(id="total-goals", className="metric-value"),
                html.Div("Total Goals Under 23", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}),
                html.Div(id="total-goals-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}),
            ], className="metric-card"),

            html.Div([
                html.Div("Total Assists", className="metric-title"),
                html.Div(id="total-assists", className="metric-value"),
                html.Div("Total Assists Under 23", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}),
                html.Div(id="total-assists-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}),
            ], className="metric-card"),

            html.Div([
                html.Div("Total Minutes", className="metric-title"),
                html.Div(id="total-minutes", className="metric-value"),
                html.Div("Total Minutes Under 23", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}),
                html.Div(id="total-minutes-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}),
            ], className="metric-card"),
        ], className="metrics-row"),
        
        # Shooting Stats Summary Metrics
        html.Div([
            html.Div([
                html.Div("Total Shots", className="metric-title"),
                html.Div(id="total-shots", className="metric-value"),
                html.Div("U23 Shots", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}),
                html.Div(id="total-shots-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}),
            ], className="metric-card"),

            html.Div([
                html.Div("Shots on Target", className="metric-title"),
                html.Div(id="total-sot", className="metric-value"),
                html.Div("U23 SoT", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}),
                html.Div(id="total-sot-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}),
            ], className="metric-card"),

            html.Div([
                html.Div("Avg SoT%", className="metric-title"),
                html.Div(id="avg-sot-pct", className="metric-value"),
                html.Div("U23 SoT%", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}),
                html.Div(id="avg-sot-pct-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}),
            ], className="metric-card"),

            html.Div([
                html.Div("Avg Goals/Shot", className="metric-title"),
                html.Div(id="avg-g-per-sh", className="metric-value"),
                html.Div("U23 G/Sh", className="metric-title", style={'fontSize': '0.8em', 'color': '#718096'}),
                html.Div(id="avg-g-per-sh-u23", className="metric-value", style={'fontSize': '1.2em', 'color': '#3182ce'}),
            ], className="metric-card"),
        ], className="metrics-row"),
        
        # Filters
        html.Div([
            html.Div([
                html.Label("Position:", style={'marginRight': '10px'}),
                dcc.Dropdown(
                    id="position-filter",
                    options=[{"label": "All Positions", "value": "all"}] +
                        [{"label": pos, "value": pos} for pos in 
                        merged_df["Pos"].unique() if pd.notna(pos)] if "Pos" in merged_df.columns else [],
                    value="all",
                    style={'width': '200px'}
                ),
                
                html.Label("Min Minutes:", style={'marginLeft': '15px', 'marginRight': '10px'}),
                dcc.Dropdown(
                    id="min-minutes-filter",
                    options=[
                        {"label": "All", "value": 0},
                        {"label": "90+", "value": 90},
                        {"label": "270+", "value": 270},
                        {"label": "450+", "value": 450},
                        {"label": "900+", "value": 900},
                    ],
                    value=0,
                    style={'width': '150px'}
                ),
                html.Label("Under 23 Only:", style={'marginLeft': '15px', 'marginRight': '10px'}),
                dcc.Checklist(
                    id="u23-filter",
                    options=[
                        {"label": "Yes", "value": "yes"},
                    ],
                    value=[],
                    style={'width': '50px'}
                ),
            ], className="filter-group"),
            
            html.Div([
                dcc.Input(
                    id="player-search",
                    type="text",
                    placeholder="Search players...",
                    style={'padding': '8px 12px', 'width': '200px', 'borderRadius': '4px', 'border': '1px solid #e2e8f0'}
                ),
            ], className="filter-group"),
        ], className="filter-container"),

        # Charts - First Row
        html.Div([
            html.Div([
                dcc.Graph(id="goals-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="assisters-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="minutes-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="teams-chart")
            ], className="chart-container"),
        ], className="dashboard-container"),
        
        # Charts - Second Row (Shooting)
        html.Div([
            html.Div([
                dcc.Graph(id="shots-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="sot-pct-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="shots-per-90-chart")
            ], className="chart-container"),
            
            html.Div([
                dcc.Graph(id="g-per-shot-chart")
            ], className="chart-container"),
        ], className="dashboard-container"),
        
        # Players Table
        html.Div([
            html.H2("Player Statistics"),
            dash_table.DataTable(
                id="players-table",
                style_table={'overflowX': 'auto'},
                style_header={
                    'backgroundColor': 'rgb(240, 240, 240)',
                    'fontWeight': 'bold',
                    'border': '1px solid #e2e8f0'
                },
                style_cell={
                    'padding': '10px',
                    'border': '1px solid #e2e8f0',
                    'textAlign': 'left'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 250, 252)'
                    }
                ]
            )
        ], className="table-container"),
        
        # Shooting Stats Table
        html.Div([
            html.H2("Player Shooting Statistics"),
            dash_table.DataTable(
                id="shooting-table",
                style_table={'overflowX': 'auto'},
                style_header={
                    'backgroundColor': 'rgb(240, 240, 240)',
                    'fontWeight': 'bold',
                    'border': '1px solid #e2e8f0'
                },
                style_cell={
                    'padding': '10px',
                    'border': '1px solid #e2e8f0',
                    'textAlign': 'left'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 250, 252)'
                    }
                ]
            )
        ], className="table-container"),
        
    ], id="content-overview", className="tab-content active"),
    
    # Content for other tabs would go here
    
], style={
    'fontFamily': 'Segoe UI, Tahoma, Geneva, Verdana, sans-serif',
    'margin': '0',
    'padding': '20px',
    'backgroundColor': '#f7fafc',
    'color': '#2d3748',
})

# Add CSS
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>A-League Analytics Dashboard</title>
        {%favicon%}
        {%css%}
        <style>
            :root {
                --primary: #3182ce;
                --secondary: #e53e3e;
                --accent: #38a169;
                --background: #f7fafc;
                --card-bg: #ffffff;
                --text: #2d3748;
                --text-light: #718096;
                --border: #e2e8f0;
                --shadow: 0 4px 6px rgba(0, 0, 0, 0.05);
            }
            
            .tabs {
                display: flex;
                border-bottom: 1px solid var(--border);
                margin-bottom: 20px;
            }
            
            .tab {
                padding: 10px 20px;
                cursor: pointer;
                border-bottom: 2px solid transparent;
                font-weight: 500;
            }
            
            .tab.active {
                border-bottom: 2px solid var(--primary);
                color: var(--primary);
            }
            
            .tab-content {
                display: none;
            }
            
            .tab-content.active {
                display: block;
            }
            
            .metrics-row {
                display: grid;
                grid-template-columns: repeat(4, 1fr);
                gap: 20px;
                margin-bottom: 20px;
            }
            
            .metric-card {
                background-color: var(--card-bg);
                border-radius: 8px;
                padding: 15px;
                box-shadow: var(--shadow);
                text-align: center;
            }
            
            .metric-value {
                font-size: 24px;
                font-weight: 700;
                color: var(--primary);
                margin: 10px 0;
            }
            
            .metric-title {
                font-size: 14px;
                color: var(--text-light);
            }
            
            .dashboard-container {
                display: grid;
                grid-template-columns: repeat(2, 1fr);
                gap: 20px;
                margin-bottom: 30px;
            }
            
            .chart-container {
                background-color: var(--card-bg);
                border-radius: 8px;
                padding: 20px;
                box-shadow: var(--shadow);
            }
            
            .table-container {
                background-color: var(--card-bg);
                border-radius: 8px;
                padding: 20px;
                box-shadow: var(--shadow);
                overflow-x: auto;
                margin-bottom: 30px;
            }
            
            .filter-container {
                display: flex;
                justify-content: space-between;
                align-items: center;
                margin-bottom: 15px;
                flex-wrap: wrap;
                gap: 10px;
            }
            
            .filter-group {
                display: flex;
                align-items: center;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# Function to get color for team
def get_team_color(team):
    # Default color if team not found
    default_color = '#808080'  # Gray
    return team_colors.get(team, default_color)

# Remove debug info callback
@app.callback(
    dash.dependencies.Output('debug-container', 'style'),
    [dash.dependencies.Input('remove-debug', 'n_clicks')]
)
def remove_debug(n_clicks):
    if n_clicks:
        return {'display': 'none'}
    return {'border': '1px solid red', 'padding': '10px', 'margin-bottom': '20px'}

# JavaScript for tab switching
app.clientside_callback(
    """
    function(n_clicks) {
        // Get all tabs and tab contents
        const tabs = document.querySelectorAll('.tab');
        const contents = document.querySelectorAll('.tab-content');
        
        // Get the clicked tab
        const clickedTab = document.querySelector('.tab:hover');
        if (!clickedTab) return;
        
        // Get the target tab content id
        const tabId = clickedTab.getAttribute('data-tab');
        const contentId = "content-" + tabId;
        
        // Remove active class from all tabs and contents
        tabs.forEach(tab => tab.classList.remove('active'));
        contents.forEach(content => content.classList.remove('active'));
        
        // Add active class to clicked tab and corresponding content
        clickedTab.classList.add('active');
        document.getElementById(contentId)?.classList.add('active');
    }
    """,
    dash.dependencies.Output('tab-overview', 'className'),
    [dash.dependencies.Input('tab-overview', 'n_clicks'),
     dash.dependencies.Input('tab-scorers', 'n_clicks'),
     dash.dependencies.Input('tab-assisters', 'n_clicks'),
     dash.dependencies.Input('tab-minutes', 'n_clicks'),
     dash.dependencies.Input('tab-shooting', 'n_clicks')]
)

# Function to open the browser automatically
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

# Run this after the app
if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run(debug=False)

Starting dashboard initialization...
Standard stats loaded successfully: 346 rows
Standard stats columns: ['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK', 'Matches']
Shooting stats loaded successfully: 346 rows
Shooting stats columns: ['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'PK', 'PKatt', 'Matches']
Processing shooting data...
First 3 rows of shooting data after processing header:
   Rk              Player  Nation Pos     Squad     Age    Born  90s  Gls  Sh  \
0   1  Alaat Abdul-Rahman     NaN  FW  W Sydney     NaN     NaN  0.0    0   0   
1   2     Asumah Abubakar  pt POR  FW  Brisbane  27-326  1997.0  3.5    1   7   
2   3       Néicer Acosta  ec ECU  MF  Brisbane  22-110  2002.0  1.4    0   5   

   SoT  SoT%  Sh/90  SoT/90  G/Sh  G/SoT  Dist  